# hw3 - ensembles

## 1 Подготовка данных

Загрузите и предобработайте данные (по своему усмотрению) из hw1

In [18]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline



In [19]:
X_train = pd.read_csv('train_features_with_answers.csv')
X_test = pd.read_csv('X_test.csv')

In [20]:
X_train.columns

Index(['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu',
       'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime',
       'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery',
       'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc',
       'Walc', 'health', 'absences', 'G3'],
      dtype='object')

In [21]:
def filling_nan(X_train):
    missing_data = X_train.isnull().sum()
    missing_results = []

    for column, missing_count in missing_data.items():
        if missing_count > 0:
            if X_train[column].dtype in ['float64', 'int64']:
                # для числовых медиана
                fill_value = X_train[column].median()
            else:
                # для категориальных мода
                fill_value = X_train[column].mode()[0]

            X_train[column] = X_train[column].fillna(fill_value)
            missing_results.append(
                f"{column} - {missing_count} - Заменено на {'медиану' if X_train[column].dtype in ['float64', 'int64'] else 'моду'} ({fill_value})"
            )
        # else:
        #     missing_results.append(f"{column} - {missing_count} - Пропусков нет")
        
    X_train = X_train[(X_train['age'] >= 15) & (X_train['age'] <= 22) & (X_train['sex'].isin(['M', 'F']))]
    print(*missing_results, sep='\n')
    print("\n")
    return X_train


In [22]:
# X_train = X_train[(X_train['age'] >= 15) & (X_train['age'] <= 22) & (X_train['sex'].isin(['M', 'F']))]

In [23]:
# categorical_columns = X_train.select_dtypes(include=['object']).columns.tolist()
# X_train = pd.get_dummies(X_train, columns=categorical_columns, drop_first=True)

# print(X_train.isnull().sum().sum())

## 2 Обоснуйте выбор слабых (базовых) алгоритмов

1. **KNN**: 
    - не требует обучения модели
    - работает хорошо для небольших наборов данных
    - чувствителен к размерности данных и может быть медленным на больших выборках.

2. **Naive Bayes**: 
    - в основном для текстовых и категориальных данных
    - предполагает независимость признаков
    - подходит для задач классификации

3. **Linear Regression**:
    - легко интерпретируемый
    - используется для регрессии с линейной зависимостью между признаками
    - хорош для начального анализа и базового прогнозирования

4. **SVM (Support Vector Machine)**: 
    - силен при работе с линейно и нелинейно разделимыми данными (с использованием ядра)
    - эффективен для небольших выборок с четкими границами классов
    - требует больших ресурсов на больших выборках

## 3 Постройте решение на основе подхода Blending

Правила:
- Нужно использовать вероятности
- Предложите что-то лучше, чем брать среднее от предсказаний моделей (оценивать уверенность алгоритмов, точности и т.д.)
- Заставьте базовые алгоритмы быть некорелированными
- Добавьте рандома (например, стройте ваши алгоритмы на разных выборках, по разному предобрабатывайте данные или применяйте для разных признаков соответствующие алгоритмы ... )
- Проявите смекалку
- Цель: метрика MSE на тесте меньше 10

In [24]:
X_train = pd.read_csv('train_features_with_answers.csv')

X_train = filling_nan(X_train)

y = X_train['G3']
X = X_train.drop(columns=['G3'])

categorical_columns = X.select_dtypes(include=['object']).columns
le = LabelEncoder()
for col in categorical_columns:
    X[col] = le.fit_transform(X[col])

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

age - 28 - Заменено на медиану (17.0)
address - 5 - Заменено на моду (U)




In [25]:
models = {
    "GaussianNB": GaussianNB(),
    "RandomForest": RandomForestRegressor(n_estimators=100, random_state=42),
    "KNN": KNeighborsRegressor(n_neighbors=5),
    "SVR": SVR(kernel='rbf', C=10, gamma='scale')
}

predictions_train = []
predictions_valid = []

for name, model in models.items():
    X_train_sample, _, y_train_sample, _ = train_test_split(X_train, y_train, test_size=0.5, random_state=42)

    model.fit(X_train_sample, y_train_sample)

    predictions_train.append(model.predict(X_train))
    predictions_valid.append(model.predict(X_valid))

predictions_train = np.array(predictions_train).T
predictions_valid = np.array(predictions_valid).T

weights = []
for preds in predictions_train.T:
    mse = mean_squared_error(y_train, preds)
    weights.append(1 / mse)

weights = np.array(weights) / np.sum(weights)

final_preds = np.dot(predictions_valid, weights)

mse_score = mean_squared_error(y_valid, final_preds)
print(f"MSE на валидации:: {mse_score:.4f}")

MSE на валидации:: 5.5624


In [26]:
final_model = RandomForestRegressor(n_estimators=100, random_state=55)
final_model.fit(predictions_train, y_train)

final_val_predictions = final_model.predict(predictions_valid)
final_val_mse = mean_squared_error(y_valid, final_val_predictions)
print(f'MSE на валидации (с метамоделью): {final_val_mse:.4f}')

MSE на валидации (с метамоделью): 7.0020


In [27]:
X_test = pd.read_csv('X_test.csv')
X_test = filling_nan(X_test)
for col in categorical_columns:
    X_test[col] = le.fit_transform(X_test[col])

predictions_test = []
for name, model in models.items():
    predictions_test.append(model.predict(X_test))

predictions_test = np.array(predictions_test).T
final_test_preds = final_model.predict(predictions_test)


In [28]:
submission = pd.read_csv('sample_submission.csv', index_col='id')
submission['G3'] = np.round(final_test_preds).astype(int)
submission.to_csv('blending_submission.csv')
submission.head()

,G3
id,
0,13
1,14
2,13
3,14
4,10


## 4 Постройте решение на основе подхода Stacking

Правила:
- Реализуйте пайплайн обучения и предсказания (например, sklearn.pipeline или класс)
- Проведите оптимизацию пайплайна
- Оцените вклад каждого базового алгоритма в итоговое предсказание
- Цель: метрика MSE на тесте меньше 10

In [29]:
X_train = pd.read_csv('train_features_with_answers.csv')
X_train = filling_nan(X_train)

y = X_train['G3']
X = X_train.drop(columns=['G3'])

categorical_columns = X.select_dtypes(include=['object']).columns
numerical_columns = X.select_dtypes(include=['int64', 'float64']).columns

X = pd.get_dummies(X, columns=categorical_columns, drop_first=True)

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

age - 28 - Заменено на медиану (17.0)
address - 5 - Заменено на моду (U)




In [30]:
base_learners = [
    ('rf', RandomForestRegressor(n_estimators=100, random_state=42)),
    ('svr', SVR(kernel='rbf', C=10))
]

meta_model = LinearRegression()

stacking_model = StackingRegressor(estimators=base_learners, final_estimator=meta_model)

pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('stacking', stacking_model)
])

pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_valid)

mse_score = mean_squared_error(y_valid, y_pred)
print(f"MSE на валидации: {mse_score:.4f}")

MSE на валидации: 6.2071


In [31]:
total_mse = 0
mse_dict = {}

for name, model in base_learners:
    model.fit(X_train, y_train)
    y_pred = model.predict(X_valid)
    mse = mean_squared_error(y_valid, y_pred)
    mse_dict[name] = mse
    total_mse += mse
    print(f"MSE для {name.upper()}: {mse:.4f}")

MSE для RF: 6.9040
MSE для SVR: 5.2219


In [32]:
for name, mse in mse_dict.items():
    contribution = (1 - mse / total_mse)
    print(f"Вес {name.upper()}: {contribution:.2%}")

Вес RF: 43.06%
Вес SVR: 56.94%


In [ ]:
X_test = pd.read_csv('X_test.csv')
X_test = pd.get_dummies(X_test, columns=categorical_columns, drop_first=True)

for col in X_train.columns:
    if col not in X_test.columns:
        X_test[col] = 0
X_test = X_test[X_train.columns]

y_test_pred = pipeline.predict(X_test)

In [34]:
submission = pd.read_csv('sample_submission.csv', index_col='id')
submission['G3'] = np.round(y_test_pred).astype(int)
submission.to_csv('stacking_submission.csv')
submission.head()

,G3
id,
0,12
1,13
2,13
3,13
4,12


## * Доп задание (не обязательно, но решение будет поощряться)

Правила:
- Постройте несколько сильных алгоритмов разного класса (это может быть бустинг, нейросеть, ансамбль слабых алгоритмов, алгоритм на статистике, что придумаете)
- Реализуйте "управляющий" алгоритм, который на основе входных данных будет выбирать, какой из  сильных алгоритмов запустить (не на основе их работы, а именно на основе данных)